In [4]:
# 对 Stomach_Cancer.csv 和 Liver_Cancer.csv 做同样的文本清洗（与 alldata 相同：正则去 DOI/序号 → 小写去标点数字 → NLTK 停用词）
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

try:
    stop_words = set(stopwords.words('english'))
except:
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

def clean_text(text):
    if pd.isna(text) or not isinstance(text, str):
        return ''
    s = text.strip()
    s = s.replace('"', '')
    s = re.sub(r'^\s*\d+\.\s*', '', s)
    s = re.sub(r'\b10\.\d{4}/[^\s]+', '', s)
    s = s.strip().lower()
    s = re.sub(r'[^a-z\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    words = [w for w in s.split() if w not in stop_words and len(w) > 0]
    return ' '.join(words)

for fname in ['Stomach_Cancer.csv', 'Liver_Cancer.csv']:
    df = pd.read_csv(fname, encoding='utf-8', on_bad_lines='skip')
    text_col = 'text' if 'text' in df.columns else ('摘要' if '摘要' in df.columns else None)
    if text_col is not None:
        df['cleaned_text'] = df[text_col].astype(str).apply(clean_text)
        df = df.drop(columns=[text_col], errors='ignore')
    # 第一列改为序号 1, 2, 3, ...
    first_col = df.columns[0]
    df[first_col] = range(1, len(df) + 1)
    df.rename(columns={first_col: '序号'}, inplace=True)
    df.to_csv(fname, index=False, encoding='utf-8')
    print(f'{fname}: 已写入 cleaned_text 列并保存，第一列为序号，共 {len(df)} 行')

Stomach_Cancer.csv: 已写入 cleaned_text 列并保存，第一列为序号，共 200 行
Liver_Cancer.csv: 已写入 cleaned_text 列并保存，第一列为序号，共 200 行


In [5]:
# Liver_Cancer.csv：删除 170 词以下的行，再从剩余行中只保留 200 行
import pandas as pd

df = pd.read_csv('Liver_Cancer.csv', encoding='utf-8', on_bad_lines='skip')
counts = df['cleaned_text'].fillna('').astype(str).str.split().str.len()
df = df[counts > 170]   # 只保留 170 词以上的行
n_after_filter = len(df)
df = df.head(200)       # 只留 200 行（取前 200 行）
df['序号'] = range(1, len(df) + 1)
df.to_csv('Liver_Cancer.csv', index=False, encoding='utf-8')

print(f'170 词以上共 {n_after_filter} 行，已保留前 200 行并保存。')

170 词以上共 200 行，已保留前 200 行并保存。


In [6]:
# Stomach_Cancer.csv：只保留 120 词以上的 200 行，其余删除
import pandas as pd

df = pd.read_csv('Stomach_Cancer.csv', encoding='utf-8', on_bad_lines='skip')
counts = df['cleaned_text'].fillna('').astype(str).str.split().str.len()
df = df[counts > 120]   # 只保留 120 词以上的行（共 200 行）
df['序号'] = range(1, len(df) + 1)
df.to_csv('Stomach_Cancer.csv', index=False, encoding='utf-8')

print(f'已保留 120 词以上的 {len(df)} 行并保存。')

已保留 120 词以上的 200 行并保存。


In [ ]:
# alldata_1_for_kaggle 2.csv：删除低于 500 词的行
import pandas as pd

fname = 'alldata_1_for_kaggle 2.csv'
df = pd.read_csv(fname, encoding='utf-8', encoding_errors='replace', on_bad_lines='skip')
text_col = df.columns[2]
counts = df[text_col].fillna('').astype(str).str.split().str.len()
df = df[counts >= 500]
df[df.columns[0]] = range(1, len(df) + 1)
df.to_csv(fname, index=False, encoding='utf-8')

print(f'已删除低于 500 词的行，保留 {len(df)} 行并保存。')
print(df[df.columns[1]].value_counts().sort_index())

已删除低于 500 词的行，保留 1154 行并保存。
label
Colon_Cancer      395
Lung_Cancer       359
Thyroid_Cancer    400
Name: count, dtype: int64


In [ ]:
# alldata_1_for_kaggle 2.csv：第三列每行只保留前 1000 词
import pandas as pd

fname = 'alldata_1_for_kaggle 2.csv'
df = pd.read_csv(fname, encoding='utf-8', encoding_errors='replace', on_bad_lines='skip')
text_col = df.columns[2]
df[text_col] = df[text_col].fillna('').astype(str).apply(lambda s: ' '.join(s.split()[:1000]))
df.to_csv(fname, index=False, encoding='utf-8')

print(f'第三列每行已截断为最多 1000 词，已保存。共 {len(df)} 行')

第三列每行已截断为最多 1000 词，已保存。共 1154 行


In [ ]:
# alldata_1_for_kaggle 2.csv：每个类别只保留 400 行
import pandas as pd

fname = 'alldata_1_for_kaggle 2.csv'
df = pd.read_csv(fname, encoding='utf-8', encoding_errors='replace', on_bad_lines='skip')
label_col = df.columns[1]
df = df.groupby(label_col, group_keys=False).apply(lambda g: g.head(400)).reset_index(drop=True)
df[df.columns[0]] = range(1, len(df) + 1)
df.to_csv(fname, index=False, encoding='utf-8')

print(f'每个类别已保留 400 行，共 {len(df)} 行并保存。')
print(df[label_col].value_counts().sort_index())

每个类别已保留 400 行，共 1154 行并保存。
label
Colon_Cancer      395
Lung_Cancer       359
Thyroid_Cancer    400
Name: count, dtype: int64


/var/folders/8x/gqvljhg13m12gzcwf08svj3m0000gn/T/ipykernel_29128/3010071658.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(label_col, group_keys=False).apply(lambda g: g.head(400)).reset_index(drop=True)


In [ ]:
# alldata_1_for_kaggle 2.csv：文本清洗（小写、去标点数字特殊字符、去 NLTK 停用词）
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

try:
    stop_words = set(stopwords.words('english'))
except:
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

def clean_text(text):
    if pd.isna(text) or not isinstance(text, str):
        return ''
    s = text.strip().lower()
    s = re.sub(r'[^a-z\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    words = [w for w in s.split() if w not in stop_words and len(w) > 0]
    return ' '.join(words)

fname = 'alldata_1_for_kaggle 2.csv'
df = pd.read_csv(fname, encoding='utf-8', encoding_errors='replace', on_bad_lines='skip')
text_col = df.columns[2]
df[text_col] = df[text_col].fillna('').astype(str).apply(clean_text)
df.to_csv(fname, index=False, encoding='utf-8')

print('第三列已清洗：小写、去标点/数字/特殊字符、去 NLTK 停用词，已保存。')
print('共', len(df), '行')
print('示例前 80 字:', df[text_col].iloc[0][:80], '...')

第三列已清洗：小写、去标点/数字/特殊字符、去 NLTK 停用词，已保存。
共 1154 行
示例前 80 字: bacteroides fragilis b fragilis produce biofilm colonisation intestinal tract ca ...


In [ ]:
# alldata_1_for_kaggle 2.csv：列名改成与 Liver_Cancer.csv 一致（序号, label, cleaned_text）
import pandas as pd

fname = 'alldata_1_for_kaggle 2.csv'
df = pd.read_csv(fname, encoding='utf-8', encoding_errors='replace', on_bad_lines='skip')
df.columns = ['序号', 'label', 'cleaned_text']
df.to_csv(fname, index=False, encoding='utf-8')

print('列名已改为：序号, label, cleaned_text，已保存。')
print('当前列名:', list(df.columns))

列名已改为：序号, label, cleaned_text，已保存。
当前列名: ['序号', 'label', 'cleaned_text']


In [ ]:
# alldata_1_for_kaggle 2.csv：按 label 分成三个新文档
import pandas as pd

fname = 'alldata_1_for_kaggle 2.csv'
df = pd.read_csv(fname, encoding='utf-8', encoding_errors='replace', on_bad_lines='skip')
label_col = 'label' if 'label' in df.columns else df.columns[1]

for label in df[label_col].unique():
    sub = df[df[label_col] == label].copy()
    sub['序号'] = range(1, len(sub) + 1)
    out_name = f'{label}.csv'
    sub.to_csv(out_name, index=False, encoding='utf-8')
    print(f'{out_name}: {len(sub)} 行')

print('三个类别已分别保存为 Colon_Cancer.csv, Lung_Cancer.csv, Thyroid_Cancer.csv')

Colon_Cancer.csv: 395 行
Lung_Cancer.csv: 359 行
Thyroid_Cancer.csv: 400 行
三个类别已分别保存为 Colon_Cancer.csv, Lung_Cancer.csv, Thyroid_Cancer.csv


In [ ]:
# 将 livercancer.csv、stomachcancer.csv 改名为与 Lung_Cancer.csv 同格式（X_Cancer.csv）
import os

renames = [
    ('livercancer.csv', 'Liver_Cancer.csv'),
    ('stomachcancer.csv', 'Stomach_Cancer.csv'),
]
for old_name, new_name in renames:
    if os.path.exists(old_name):
        os.rename(old_name, new_name)
        print(f'{old_name} → {new_name}')
    else:
        print(f'未找到 {old_name}，跳过')
print('改名完成。')

未找到 livercancer.csv，跳过
未找到 stomachcancer.csv，跳过
改名完成。


In [ ]:
# 五个 CSV 第三列分别删除对应病名词（整词）
import pandas as pd

config = [
    ('Colon_Cancer.csv', {'colon', 'cancer', 'cancers'}),
    ('Liver_Cancer.csv', {'liver', 'cancer', 'cancers'}),
    ('Lung_Cancer.csv', {'lung', 'cancer', 'cancers'}),
    ('Stomach_Cancer.csv', {'stomach', 'cancer', 'cancers'}),
    ('Thyroid_Cancer.csv', {'thyroid', 'cancer', 'cancers'}),
]

for fname, remove_words in config:
    try:
        df = pd.read_csv(fname, encoding='utf-8', on_bad_lines='skip')
    except FileNotFoundError:
        print(f'未找到 {fname}，跳过')
        continue
    text_col = df.columns[2]
    df[text_col] = df[text_col].fillna('').astype(str).apply(
        lambda s: ' '.join(w for w in s.split() if w not in remove_words)
    )
    df.to_csv(fname, index=False, encoding='utf-8')
    print(f'{fname}: 已删除 {remove_words}，已保存。')

print('五个文件第三列病名词已删除完成。')

Colon_Cancer.csv: 已删除 {'cancers', 'colon', 'cancer'}，已保存。
Liver_Cancer.csv: 已删除 {'liver', 'cancer', 'cancers'}，已保存。
Lung_Cancer.csv: 已删除 {'lung', 'cancer', 'cancers'}，已保存。
Stomach_Cancer.csv: 已删除 {'cancers', 'cancer', 'stomach'}，已保存。
Thyroid_Cancer.csv: 已删除 {'cancers', 'cancer', 'thyroid'}，已保存。
五个文件第三列病名词已删除完成。


In [ ]:
# 统计五个文件第三列每行的词数（按空格分词）
import pandas as pd

files = ['Colon_Cancer.csv', 'Liver_Cancer.csv', 'Lung_Cancer.csv', 'Stomach_Cancer.csv', 'Thyroid_Cancer.csv']
text_col_name = 'cleaned_text'

rows = []
for fname in files:
    try:
        df = pd.read_csv(fname, encoding='utf-8', on_bad_lines='skip')
    except FileNotFoundError:
        print(f'未找到 {fname}，跳过')
        continue
    col = text_col_name if text_col_name in df.columns else df.columns[2]
    # 第三列每行的词数
    counts = df[col].fillna('').astype(str).str.split().str.len()
    rows.append({
        '文件': fname,
        '行数': len(df),
        '第三列每行最少(词)': int(counts.min()),
        '第三列每行最多(词)': int(counts.max()),
        '第三列每行平均(词)': round(counts.mean(), 1),
        '第三列每行中位数(词)': int(counts.median()),
    })
    print(f'{fname}: 第三列每行词数 — 最少={counts.min()}, 最多={counts.max()}, 平均={counts.mean():.1f}')

summary = pd.DataFrame(rows)
print('\n汇总表（第三列每行的词数）:')
print(summary.to_string(index=False))

Colon_Cancer.csv: 第三列每行词数 — 最少=557, 最多=776, 平均=663.0
Liver_Cancer.csv: 第三列每行词数 — 最少=133, 最多=394, 平均=207.9
Lung_Cancer.csv: 第三列每行词数 — 最少=453, 最多=860, 平均=625.0
Stomach_Cancer.csv: 第三列每行词数 — 最少=96, 最多=517, 平均=146.2
Thyroid_Cancer.csv: 第三列每行词数 — 最少=590, 最多=776, 平均=661.4

汇总表（第三列每行的词数）:
                文件  行数  第三列每行最少(词)  第三列每行最多(词)  第三列每行平均(词)  第三列每行中位数(词)
  Colon_Cancer.csv 395         557         776       663.0          662
  Liver_Cancer.csv 200         133         394       207.9          197
   Lung_Cancer.csv 359         453         860       625.0          624
Stomach_Cancer.csv 200          96         517       146.2          136
Thyroid_Cancer.csv 400         590         776       661.4          656


In [ ]:
# 五个文件：第三列截断到 96 词，每个文件只保留 200 行
import pandas as pd

files = ['Colon_Cancer.csv', 'Liver_Cancer.csv', 'Lung_Cancer.csv', 'Stomach_Cancer.csv', 'Thyroid_Cancer.csv']
text_col_name = 'cleaned_text'

for fname in files:
    try:
        df = pd.read_csv(fname, encoding='utf-8', on_bad_lines='skip')
    except FileNotFoundError:
        print(f'未找到 {fname}，跳过')
        continue
    col = text_col_name if text_col_name in df.columns else df.columns[2]
    # 第三列每行只保留前 96 词
    df[col] = df[col].fillna('').astype(str).apply(lambda s: ' '.join(s.split()[:96]))
    # 只保留前 200 行
    df = df.head(200)
    # 序号从 1 开始
    df[df.columns[0]] = range(1, len(df) + 1)
    df.to_csv(fname, index=False, encoding='utf-8')
    print(f'{fname}: 第三列已截断为 96 词，保留 {len(df)} 行，已保存。')

print('五个文件处理完成。')

Colon_Cancer.csv: 第三列已截断为 96 词，保留 200 行，已保存。
Liver_Cancer.csv: 第三列已截断为 96 词，保留 200 行，已保存。
Lung_Cancer.csv: 第三列已截断为 96 词，保留 200 行，已保存。
Stomach_Cancer.csv: 第三列已截断为 96 词，保留 200 行，已保存。
Thyroid_Cancer.csv: 第三列已截断为 96 词，保留 200 行，已保存。
五个文件处理完成。
